In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
import h5py
import keras
from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.optimizers import *
from keras.regularizers import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

KFOLD_SEED = 42


def rtb_confusion_matrix(test_labels, y_preds):
    m = confusion_matrix(test_labels, y_preds)
    
    print("================================")
    print("Confusion Matrix:")
    print("True Negative = %d" % m[0][0])
    print("False Negative = %d" % m[1][0])
    print("True Positive = %d" % m[1][1])
    print("False Positive = %d" % m[0][1])


def rtb_f1_score(test_labels, y_preds):
    f = f1_score(test_labels, y_preds)
    print("================================")
    print("f1 score = %0.3f" % f)


def print_metrics(true_labels, y_preds, y_scores, is_train=True):
    if is_train:
        print("--------train---------")
    else:
        print("--------test---------")
    
    rtb_confusion_matrix(true_labels, y_preds)
    rtb_f1_score(true_labels, y_preds)
    print("================================")
    print("ROC AUC Score = %0.3f" % roc_auc_score(true_labels, y_scores.argmax(axis=-1)))

In [2]:
input_path = '~/data/biddings.csv'
data = pd.read_csv(input_path)
print(data.shape)

train = data[:800000]
test = data[800000:]

sample = train.sample(frac=1)
features = sample.drop('convert', axis=1).values
labels = sample.convert.ravel()

test_features = test.drop('convert', axis=1).values
test_labels = test.convert.ravel()

(1000000, 89)


In [12]:
lr = LogisticRegression(penalty='l2', random_state=KFOLD_SEED, verbose=2)

model = lr.fit(features, labels)
predicted_scores = model.predict_proba(test_features)
predicted_labels = model.predict(test_features)
print(predicted_scores.shape, predicted_labels.shape)

print_metrics(test_labels, predicted_labels, predicted_scores, is_train=False)


[LibLinear](200000, 2) (200000,)
--------test---------
Confusion Matrix:
True Negative = 199623
False Negative = 377
True Positive = 0
False Positive = 0
f1 score = 0.000
ROC AUC Score = 0.500


/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
def logistic_regression():
    lr = LogisticRegression(penalty='l2', random_state=KFOLD_SEED, verbose=2)
    return lr
    

def ensembler_test(classifier_fn, ensemblers):
    rus = RandomUnderSampler(ratio={0: 1531*10, 1: 1531}, random_state=KFOLD_SEED)
    X_us, y_us = rus.fit_sample(features, labels)
    
    for i, e in enumerate(ensemblers):
        print("fitting sample")
        X_res, y_res = e.fit_sample(X_us, y_us)
        print(X_res.shape, y_res.shape)
        clf = classifier_fn()
        print("training")
        
        for j, X_train in enumerate(X_res):
            model = clf.fit(X_train, y_res[j])
        
        predicted_scores = model.predict_proba(test_features)
        predicted_labels = model.predict(test_features)
        
        print("Ensembler %d" % i)
        print_metrics(test_labels, predicted_labels, predicted_scores, is_train=False)
    
bc = BalanceCascade(random_state=KFOLD_SEED)

dt = DecisionTreeClassifier(random_state=KFOLD_SEED)
bc_dt = BalanceCascade(estimator=dt, random_state=KFOLD_SEED)

rf = RandomForestClassifier(n_jobs=-1, random_state=KFOLD_SEED, verbose=1)
bc_rf = BalanceCascade(estimator=rf, random_state=KFOLD_SEED)

ensemblers = [bc, bc_dt, bc_rf]
ensembler_test(logistic_regression, ensemblers)

fitting sample
(18, 3062, 88) (18, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 117098
False Negative = 154
True Positive = 223
False Positive = 82525
f1 score = 0.005
ROC AUC Score = 0.589
fitting sample
(18, 3062, 88) (18, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 1
--------test---------
Confusion Matrix:
True Negative = 136944
False Negative = 135
True Positive = 242
False Positive = 62679
f1 score = 0.008
ROC AUC Score = 0.664
fitting sample


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_j

(15, 3062, 88) (15, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 2
--------test---------
Confusion Matrix:
True Negative = 129009
False Negative = 151
True Positive = 226
False Positive = 70614
f1 score = 0.006
ROC AUC Score = 0.623


In [19]:
print(X_res.shape, y_res.shape)

NameError: name 'X_res' is not defined